In [1]:
import csv
import pandas as pd
import itertools

## Setup

In [2]:
path_bawe_csv = '/home/simon/Downloads/BAWE_data.csv'
path_flip_csv = '/home/simon/Downloads/flip_data.csv'
path_result_dir = '/home/simon/Downloads/flipcorrelation/' # All results will be saved here. Has to be a directory. 
                                                           # Don't forget the trailing slash (/)!

## Dataset
Prepare the dataset. The source files are `BAWE_data.csv` and `flip_data.csv`. The preprocessed combined result can be 
found in the summary Google sheet. 

In [3]:
level_to_int = {
    'A1': 1,
    'A2': 2,
    'B1': 3,
    'B2': 4,
    'C1': 5,
    'C2': 6
}

grade_variables = {
    'Content_Grade_AVG': 'CONTENT',
    'Organiz_Grade_AVG': 'ORGANIZATION',
    'Vocab_Grade_AVG': 'VOCAB',
    'Grammar_Grade_AVG': 'GRAMMAR',
    'Mechanics_Grade_AVG': 'MECHANICS'
}

def combined_grade_column_names():
    for l in range(1, len(grade_variables)+1):
        for c in itertools.combinations(grade_variables.keys(), l):
            yield 'M_LEVEL_' + '_'.join([grade_variables[n] for n in c])
             
def generate_combined_grade_columns(row):
    level_numerical = level_to_int[row['Level']]
    
    for l in range(1, len(grade_variables)+1):
        for c in itertools.combinations(grade_variables.keys(), l):
            name = 'M_LEVEL_' + '_'.join([grade_variables[n] for n in c])
            grade_sum = sum([float(row[v]) for v in c])
            row[name] = ((level_numerical-1)*(4*l)) + grade_sum
            
    return row

def add_flip_columns(bawe_column, i):
    """
    Add columns which appear only in the flip dataset to the bawe dataset. At the end we'll remove any column which 
    does not appear in BOTH datasets. 
    :param bawe_column: 
    :param i: 
    :return: 
    """
    bawe_column['Observation'] = i+1
    bawe_column['ID'] = i+1
    bawe_column['Student'] = 'BAWE' + str(i+1)
    bawe_column['Session'] = ''
    bawe_column['Level_Specific'] = 21
    bawe_column['Level'] = 'C2'
    bawe_column['Level_Num'] = 6
    bawe_column['Level_Grade_Num'] = 21
    bawe_column['Content_Grade_AVG'] = 1
    bawe_column['Organiz_Grade_AVG'] = 1
    bawe_column['Vocab_Grade_AVG'] = 1
    bawe_column['Grammar_Grade_AVG'] = 1
    bawe_column['Mechanics_Grade_AVG'] = 1
    bawe_column['Overall_Grade_AVG'] = 1
    del bawe_column['']
    return bawe_column

bawe_headers = set()
with open(path_bawe_csv) as csvfile:
    csvreader = csv.DictReader(csvfile)
    bawe = [add_flip_columns(r, i) for i, r in enumerate(csvreader)]
    for h in bawe[0].keys():
        
        bawe_headers.add(h)
flip_headers = set()
with open(path_flip_csv) as csvfile:
    csvreader = csv.DictReader(csvfile)
    flip = [r for r in csvreader if r['Vocab_Grade_AVG'] != 'NA']
    for h in bawe[0].keys():
        flip_headers.add(h)
    
# Remove header which only appear in one dataset 
headers = flip_headers.intersection(bawe_headers)
data = [generate_combined_grade_columns({k: r[k] for k in r.keys() if k in headers}) for r in flip + bawe]

rawdf = pd.DataFrame.from_dict(data)
rawdf.to_csv(path_result_dir + 'raw_combined_data.csv')
rawdf

,Observation,ID,Student,Session,Level_Specific,Level,Level_Num,Level_Grade_Num,Content_Grade_AVG,Organiz_Grade_AVG,...,M_LEVEL_ORGANIZATION_VOCAB_GRAMMAR,M_LEVEL_ORGANIZATION_VOCAB_MECHANICS,M_LEVEL_ORGANIZATION_GRAMMAR_MECHANICS,M_LEVEL_VOCAB_GRAMMAR_MECHANICS,M_LEVEL_CONTENT_ORGANIZATION_VOCAB_GRAMMAR,M_LEVEL_CONTENT_ORGANIZATION_VOCAB_MECHANICS,M_LEVEL_CONTENT_ORGANIZATION_GRAMMAR_MECHANICS,M_LEVEL_CONTENT_VOCAB_GRAMMAR_MECHANICS,M_LEVEL_ORGANIZATION_VOCAB_GRAMMAR_MECHANICS,M_LEVEL_CONTENT_ORGANIZATION_VOCAB_GRAMMAR_MECHANICS
0,001__Janina_Christina_Andreß__Pre-Test.txt,1,Janina.Andress@lmu.de,Pre-Test,B1.1,B1,3,11,2,2,...,31.5,32.0,31.5,32.5,41.5,42.0,41.5,42.5,42.5,52.5
1,002__Abdelrahman_Munir_Awad_Assaf__Pre-Test.txt,2,A.Assaf@campus.lmu.de,Pre-Test,B2.1,B2,4,14,3,2,...,41.5,42.0,41.5,41.5,56.5,57.0,56.5,56.5,55.5,70.5
2,004__Annika_Bäum__Pre-Test.txt,4,A.Baeum@campus.lmu.de,Pre-Test,B2.1,B2,4,15,3.5,2.5,...,44.0,44.0,43.5,44.0,59.5,59.5,59.0,59.5,58.5,74.0
3,005__Lena_Marie_Baumann__Pre-Test.txt,5,L.Baumann@campus.lmu.de,Pre-Test,B1.1,B1,3,12,4,4,...,34.0,35.0,35.0,34.0,46.0,47.0,47.0,46.0,46.0,58.0
4,007__Tekle_Berishvili__Pre-Test.txt,7,Tekle.Berishvili@campus.lmu.de,Pre-Test,A2.1,A2,2,8,3.5,3,...,22.0,20.5,21.5,21.5,29.5,28.0,29.0,29.0,28.5,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,96,96,BAWE96,,21,C2,6,21,1,1,...,63.0,63.0,63.0,63.0,84.0,84.0,84.0,84.0,84.0,105.0
485,97,97,BAWE97,,21,C2,6,21,1,1,...,63.0,63.0,63.0,63.0,84.0,84.0,84.0,84.0,84.0,105.0
486,98,98,BAWE98,,21,C2,6,21,1,1,...,63.0,63.0,63.0,63.0,84.0,84.0,84.0,84.0,84.0,105.0
487,99,99,BAWE99,,21,C2,6,21,1,1,...,63.0,63.0,63.0,63.0,84.0,84.0,84.0,84.0,84.0,105.0


## 2. Calculate the correlation matrix
First we'll remove any columns which contain non numerical values. Then we'll calculate the correlation matrix and 
finally we'll save a sheet for each depended variable sorted by the variables which have the highest correlation with 
it.

In [4]:
# prepare for pandas
string_headers = {'Observation', 'Student', 'Session', 'Level_Specific', 'Level'}
dataPandas = [{k: float(r[k]) for k in r.keys() if k not in string_headers} for r in data]
df = pd.DataFrame.from_dict(dataPandas)

In [5]:
# Correlation
df_corr = df.corr()
df_corr.to_csv(path_result_dir + 'correlation_matrix.csv')
df_corr

,ID,Level_Num,Level_Grade_Num,Content_Grade_AVG,Organiz_Grade_AVG,Vocab_Grade_AVG,Grammar_Grade_AVG,Mechanics_Grade_AVG,Overall_Grade_AVG,Word.Count,...,M_LEVEL_ORGANIZATION_VOCAB_GRAMMAR,M_LEVEL_ORGANIZATION_VOCAB_MECHANICS,M_LEVEL_ORGANIZATION_GRAMMAR_MECHANICS,M_LEVEL_VOCAB_GRAMMAR_MECHANICS,M_LEVEL_CONTENT_ORGANIZATION_VOCAB_GRAMMAR,M_LEVEL_CONTENT_ORGANIZATION_VOCAB_MECHANICS,M_LEVEL_CONTENT_ORGANIZATION_GRAMMAR_MECHANICS,M_LEVEL_CONTENT_VOCAB_GRAMMAR_MECHANICS,M_LEVEL_ORGANIZATION_VOCAB_GRAMMAR_MECHANICS,M_LEVEL_CONTENT_ORGANIZATION_VOCAB_GRAMMAR_MECHANICS
ID,1.000000,-0.374721,-0.295229,0.534965,0.570978,0.467692,0.448113,0.460743,0.539475,-0.379718,...,-0.306231,-0.304250,-0.305871,-0.313583,-0.303765,-0.302240,-0.303490,-0.309423,-0.307557,-0.305344
Level_Num,-0.374721,1.000000,0.982546,-0.574300,-0.519982,-0.447845,-0.404500,-0.420005,-0.555242,0.499023,...,0.986522,0.985980,0.986501,0.986841,0.986263,0.985781,0.986232,0.986713,0.986648,0.986452
Level_Grade_Num,-0.295229,0.982546,1.000000,-0.425386,-0.364553,-0.290009,-0.246829,-0.262509,-0.395693,0.407213,...,0.998251,0.998361,0.998308,0.997769,0.998687,0.998699,0.998713,0.998527,0.998360,0.998722
Content_Grade_AVG,0.534965,-0.574300,-0.425386,1.000000,0.928746,0.854055,0.809963,0.841226,0.953111,-0.648201,...,-0.449346,-0.444999,-0.448864,-0.455865,-0.440404,-0.437069,-0.440041,-0.445503,-0.449869,-0.442654
Organiz_Grade_AVG,0.570978,-0.519982,-0.364553,0.928746,1.000000,0.885354,0.845772,0.873020,0.959225,-0.622953,...,-0.381745,-0.377368,-0.381309,-0.391777,-0.378452,-0.375109,-0.378120,-0.386165,-0.383144,-0.380261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M_LEVEL_CONTENT_ORGANIZATION_VOCAB_MECHANICS,-0.302240,0.985781,0.998699,-0.437069,-0.375109,-0.301081,-0.263040,-0.272103,-0.408731,0.417045,...,0.999304,0.999779,0.999446,0.998930,0.999712,1.000000,0.999804,0.999635,0.999552,0.999866
M_LEVEL_CONTENT_ORGANIZATION_GRAMMAR_MECHANICS,-0.303490,0.986232,0.998713,-0.440041,-0.378120,-0.306831,-0.260538,-0.273790,-0.411312,0.419099,...,0.999363,0.999577,0.999763,0.999089,0.999713,0.999804,1.000000,0.999713,0.999636,0.999898
M_LEVEL_CONTENT_VOCAB_GRAMMAR_MECHANICS,-0.309423,0.986713,0.998527,-0.445503,-0.386165,-0.303726,-0.259866,-0.276058,-0.414117,0.420923,...,0.999557,0.999583,0.999612,0.999694,0.999687,0.999635,0.999713,1.000000,0.999800,0.999893
M_LEVEL_ORGANIZATION_VOCAB_GRAMMAR_MECHANICS,-0.307557,0.986648,0.998360,-0.449869,-0.383144,-0.302970,-0.258955,-0.275367,-0.414447,0.422064,...,0.999802,0.999819,0.999853,0.999772,0.999612,0.999552,0.999636,0.999800,1.000000,0.999846


In [6]:
# Save results
csv_data = []
for combined_var in combined_grade_column_names():
    csv_data.append(list(df_corr[[combined_var]].abs().sort_values(combined_var, ascending=False).itertuples()))

In [20]:
csv_text = ''

#write headers
line_text = []
for name in combined_grade_column_names():
    line_text.append('')
    line_text.append(name)
    line_text.append('')
csv_text += ','.join(line_text) + '\n'

#write rows
for i in range(len(csv_data[0])):
    line_text = []
    for var_data in csv_data:
        line_text.append(str(var_data[i][0]))
        line_text.append(str(var_data[i][1]))
        line_text.append('')
    csv_text += ','.join(line_text)
    csv_text += '\n'
    

In [21]:
with open(path_result_dir + 'corr_sorted.csv', 'w') as fh:
    fh.write(csv_text)




